In [9]:
import os
import sys
import openai
__file__ = '../../'
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
sys.path.append(os.path.abspath(__file__))  # Agrega el directorio anterior al PATH

from drf import settings


In [4]:
prompt = 'Ayuda a saludar en portugues'

In [5]:
mensajes = [
    {"role": "system", "content": "Eres un asistente virtual que ayuda con preguntas de español a portugues y viceversa."},
    {"role": "user", "content": prompt},
]

In [10]:
openai.api_key = settings.API_KEY_OPENAI

respuesta = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = mensajes,
    max_tokens = 100,
    temperature = 0,
)

NameError: name 'mensajes' is not defined

In [7]:
print(respuesta.choices[0].message.content)

¡Claro! Aquí tienes algunas formas de saludar en portugués:

- Hola: Olá
- Buenos días: Bom dia
- Buenas tardes: Boa tarde
- Buenas noches: Boa noite

Espero que te sean útiles. ¿Hay algo más en lo que pueda ayudarte?


In [55]:
def obtener_completion( contexto , model = 'gpt-3.5-turbo'):
    mensaje = openai.chat.completions.create(
        model = model,
        messages = contexto,
        max_tokens = 100,
        temperature = 0,
    )
    return mensaje.choices[0].message.content

In [56]:
contexto = [
    {"role": "system", "content": "Eres un asistente virtual que ayuda con preguntas de español a portugues y viceversa."},
    {"role": "user", "content": "Ayudame a saludar en portugues"},
]

respuesta = obtener_completion(contexto)

In [57]:
print(respuesta)

¡Claro! Para saludar en portugués puedes decir "Olá" que significa "Hola" o "Bom dia" que significa "Buenos días". ¿Hay algo más en lo que pueda ayudarte?


In [58]:
# Crearemos un varaible global que contenga el contexto de la conversacion

contexto = [{"role": "assistant", "content": "Ayudame a saludar en portugues"}]

contento = contexto + [{"role": "system", "content": respuesta}]

print(contento)

[{'role': 'user', 'content': 'Ayudame a saludar en portugues'}, {'role': 'system', 'content': '¡Claro! Para saludar en portugués puedes decir "Olá" que significa "Hola" o "Bom dia" que significa "Buenos días". ¿Hay algo más en lo que pueda ayudarte?'}]


In [5]:
contexto = [
    {"role": "system", "content": "Eres un asistente virtual que ayuda con preguntas de español a portugues y viceversa."},
    {"role": "user", "content": "Ayudame a saludar en portugues"},
]

In [11]:
from groq import Groq

groq_client =Groq(
    api_key = settings.API_KEY_GROQ,
)

respuesta = groq_client.chat.completions.create(
    model = 'llama3-8b-8192',
    messages = contexto, 
    max_tokens = 300,   
    temperature = 0,
)

NameError: name 'contexto' is not defined

In [12]:
print(respuesta.choices[0].message.content)

NameError: name 'respuesta' is not defined

In [15]:
contexto = [{
            "data": [],
            "personalidad": "sad",
        }]
print(contexto[0]['data'])

[]


In [20]:
import os
import sounddevice as sd
from scipy.io.wavfile import write
from groq import Groq

def record_audio(filename, duration=5, samplerate=44100):
    print("Recording...")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=2, dtype='int16')
    sd.wait()  # Esperar a que termine la grabación
    write(filename, samplerate, audio)  # Guardar el archivo de audio
    print("Recording finished")

def transcribe_audio(filename, api_key):
    client = Groq(api_key=api_key)

    with open(filename, "rb") as file:
        #translations
        #transcriptions
        transcription = client.audio.transcriptions.create(
            file=(filename, file.read()),
            model="whisper-large-v3",
            prompt="Specify context or spelling",  # Opcional
            response_format="json",  # Opcional
            language="pt",  # Opcional
            temperature=0.0  # Opcional
        )
        return transcription.text

def main():
    audio_filename = "recorded_audio.wav"
    api_key = settings.API_KEY_GROQ  # Reemplaza con tu API key de Groq
    record_audio(audio_filename, duration=5)  # Grabar 5 segundos de audio
    transcription = transcribe_audio(audio_filename, api_key)
    print("Transcription:", transcription)

if __name__ == "__main__":
    main()

# pip install sounddevice scipy

Recording...
Recording finished
Transcription:  kawal e o seu nome


In [24]:
import os
import sounddevice as sd
from scipy.io.wavfile import write
from groq import Groq
from difflib import SequenceMatcher

def record_audio(filename, duration=5, samplerate=44100):
    print("Recording...")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=2, dtype='int16')
    sd.wait()  # Esperar a que termine la grabación
    write(filename, samplerate, audio)  # Guardar el archivo de audio
    print("Recording finished")

def transcribe_audio(filename, api_key):
    client = Groq(api_key=api_key)

    with open(filename, "rb") as file:
        transcription = client.audio.transcriptions.create(
            file=(filename, file.read()),
            model="whisper-large-v3",
            prompt="",  # Opcional
            response_format="json",  # Opcional
            language="pt",  # Cambiar a portugués
            temperature=0.0  # Opcional
        )
        return transcription.text

def compare_transcription(transcription, reference_text):
    # Usar SequenceMatcher para comparar la transcripción con el texto de referencia
    matcher = SequenceMatcher(None, transcription, reference_text)
    score = matcher.ratio()  # Obtener la similitud entre los dos textos
    return score

def evaluate_pronunciation(reference_text, duration=5):
    audio_filename = "recorded_audio.wav"
    api_key = os.getenv("API_KEY_GROQ")
    if not api_key:
        raise ValueError("La variable de entorno GROQ_API_KEY no está configurada.")

    record_audio(audio_filename, duration=duration)  # Grabar el audio
    transcription = transcribe_audio(audio_filename, api_key)  # Transcribir el audio
    print(f"Transcription: {transcription}")
    
    score = compare_transcription(transcription, reference_text)  # Comparar con el texto de referencia
    print(f"Pronunciation score: {score:.2f}")

    return score

if __name__ == "__main__":
    reference_text = "Este é o texto de referência para comparar."  # Texto de referencia en portugués
    evaluate_pronunciation(reference_text, duration=5)


Recording...
Recording finished
Transcription:  O que está acontecendo? Texto de referência para comparar.
Pronunciation score: 0.71


In [13]:
OUTPUT_DIR = BASE_DIR + "api-gateway/media/audios"
print(OUTPUT_DIR)

e:\api-rest-django/media/audios


sk_6c32b7434c09b33b117d326bcc039414e8bf5fbdbf3109220


In [35]:
import requests
import os

# Constantes para el script
CHUNK_SIZE = 1024  # Tamaño de los bloques para lectura/escritura
XI_API_KEY = '' # Tu clave API para autenticación
VOICE_ID = "33B4UnXyTNbgLmdEDh5P"  # ID del modelo de voz a ser usado
TEXTS_TO_SPEAK_T5 = [
    "Eu gosto de maçã.",
    "A banana está madura.",
    "Vamos comprar uvas.",
    "O tomate é vermelho.",
    "Eu preciso de batatas.",
    "Eu quero um suco de laranja.",
    "Vamos pedir uma pizza.",
    "O garçom é muito simpático.",
    "Eu vou tomar um café.",
    "A sobremesa está deliciosa.",
    "O sorvete está derretendo.",
    "Eu prefiro chocolate.",
    "Vamos beber água.",
    "O bolo de morango é o meu favorito.",
    "O suco de manga é refrescante.",
    "Eu adoro cozinhar.",
    "Vamos preparar uma salada.",
    "O arroz está quase pronto.",
    "Preciso de uma receita nova.",
    "Vamos assar um bolo.",
    "A feijoada é um prato típico brasileiro.",
    "Eu gosto de bacalhau à portuguesa.",
    "A moqueca é deliciosa.",
    "O churrasco é muito popular no Brasil.",
    "A caipirinha é uma bebida tradicional."
]  # Lista de textos a ser convertidos en voz
OUTPUT_DIR = BASE_DIR + "/api-gateway/media/audios/topic_5"  # Directorio para guardar los archivos de audio

# Verifica si el directorio de salida existe, si no, lo crea
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# URL para la solicitud de la API de Text-to-Speech
tts_url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID}/stream"

# Encabezados para la solicitud, incluyendo la clave API para autenticación
headers = {
    "Accept": "application/json",
    "xi-api-key": XI_API_KEY
}

# Bucle a través de las frases y hace la solicitud para cada una
for index, text in enumerate(TEXTS_TO_SPEAK_T5):
    # Datos para la solicitud, incluyendo el texto y configuraciones de voz
    data = {
        "text": text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.8,
            "style": 0.0,
            "use_speaker_boost": True
        }
    }
    
    # Hacer la solicitud POST para la API de TTS con encabezados y datos, habilitando la respuesta en streaming
    response = requests.post(tts_url, headers=headers, json=data, stream=True)
    
    # Nombre del archivo de salida
    output_path = os.path.join(OUTPUT_DIR, f"{index+1}.mp3")
    
    # Verificar si la solicitud fue exitosa
    if response.ok:
        # Abrir el archivo de salida en modo de escritura binaria
        with open(output_path, "wb") as f:
            # Leer la respuesta en bloques y escribir en el archivo
            for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
                f.write(chunk)
        # Informar al usuario que el audio fue guardado con éxito
        print(f"Audio stream saved successfully to {output_path}.")
    else:
        # Imprimir el mensaje de error si la solicitud no fue exitosa
        print(f"Failed to save audio for text: '{text}'. Error: {response.text}")


Audio stream saved successfully to e:\api-rest-django/api-gateway/media/audios/topic_5\1.mp3.
Audio stream saved successfully to e:\api-rest-django/api-gateway/media/audios/topic_5\2.mp3.
Audio stream saved successfully to e:\api-rest-django/api-gateway/media/audios/topic_5\3.mp3.
Audio stream saved successfully to e:\api-rest-django/api-gateway/media/audios/topic_5\4.mp3.
Audio stream saved successfully to e:\api-rest-django/api-gateway/media/audios/topic_5\5.mp3.
Audio stream saved successfully to e:\api-rest-django/api-gateway/media/audios/topic_5\6.mp3.
Audio stream saved successfully to e:\api-rest-django/api-gateway/media/audios/topic_5\7.mp3.
Audio stream saved successfully to e:\api-rest-django/api-gateway/media/audios/topic_5\8.mp3.
Audio stream saved successfully to e:\api-rest-django/api-gateway/media/audios/topic_5\9.mp3.
Audio stream saved successfully to e:\api-rest-django/api-gateway/media/audios/topic_5\10.mp3.
Audio stream saved successfully to e:\api-rest-django/api-g

In [22]:
import requests

XI_API_KEY = 'sk_6c32b7434c09b33b117d326bcc039414e8bf5fbdbf310922'
url = "https://api.elevenlabs.io/v1/shared-voices"

headers = {
    "xi-api-key": XI_API_KEY
}

params = {
    "language": "portuguese"
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

In [24]:
for voice in data['voices']:
    print(f"{voice['name']}; {voice['voice_id']}")

Lulu; z5Wiw7TIqo9tCUzvq4Yc
Will ; kIfcKu9kr8RZrbz7H3ox
Greg M - Natural, Conversational ; duHkTtQmj6QYfaOiXceQ
Tara - Conversational Expressive Voice; P7vsEyTOpZ6YUTulin8m
Shayne - Narrator, RJ Voice; vKnhz1CSirDNQVFqLbul
Young Jamal; 6OzrBCQf8cjERkYgzSg8
Magic Jack - Super Narrator and Story Teller; W0T33KO9BEThHKbmurx1
German Petra - English with hard accent; ztyYYqlYMny7nllhThgo
Beatrice Wandsworthy; o0JsuVZWfsLCnWUxPKOW
Tara; 4wYIYo1qwewRR49K5aAu
Paracrit; DjW94oR4PgyQ4krNpmiD
Est 2; YkCxSAVzN6J7VXgYm7WN
Christine ; 08x5lo5SPxyFD2VjQkYS
Wawi; NvlR4gnvBhch6cgZSq5G
Johnson - American Male voice ; H9rGhVjvDtFhHkh3hzVk
Philosopher Daniel; 1ZqNYaLHXm4MhZUzxaJW
Epiktet Philosoph ; qfydv1ckf7Gwp8ptlbLz
Nakiso; pC7rIWMbpc07K3a9Pa9d
Ola - African Princess; RLTG7sObICVfmJ2w1egM
Luna - Well rounded, insightful, charismatic; ICKTwCUhZRJPsgZYIvtK
Ezreal; PfqQtNzoasV6JK0UY37t
George; jpbWvwfGMsQkvqCiG2dM
Adriano - 44; F1BhJVPjP88brpBVEuKe
Adriano - narrador 43; qIiC4FBDyxdzMl6iQCtJ
Samuel; W6xDy